In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

from gluonts.dataset.pandas import PandasDataset
from gluonts.torch import DeepAREstimator, TemporalFusionTransformerEstimator
# from gluonts.trainer import Trainer
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.common import ListDataset
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

from tqdm.autonotebook import tqdm
import yfinance as yf

/home/abizard/miniconda3/envs/finetuning/lib/python3.12/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/tmp/ipykernel_7236/676843608.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
symbols = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'JPM', 'JNJ', 'V', 'PG', 'UNH', 'MA', 'INTC', 'VZ', 'HD', 'T', 'DIS', 'BAC', 'XOM', 'PFE', 'KO', 'CSCO', 'CMCSA', 'PEP', 'BA', 'WFC', 'MCD', 'COST', 'CVX', 'ABT', 'LLY', 'NKE', 'MDT', 'IBM', 'NEE', 'DHR', 'HON', 'ACN', 'TXN', 'GILD', 'LMT', 'BMY', 'SBUX', 'FIS', 'AMGN', 'MO', 'INTU', 'UNP', 'MMM', 'CHTR', 'TMO', 'LOW', 'BKNG', 'UPS', 'ADBE', 'QCOM', 'ORCL', 'MS', 'BLK', 'GS', 'NOW', 'SCHW', 'MDLZ', 'GE', 'C', 'FDX', 'CAT', 'AXP', 'SPGI', 'TGT', 'ISRG', 'DE', 'SYK', 'NSC', 'MU', 'SO', 'GM', 'ADP', 'VRTX', 'ZTS', 'USB', 'CI', 'GD', 'REGN', 'RTX', 'DUK', 'PGR', 'CL', 'CVS', 'D', 'CSX', 'TFC', 'COP', 'SPG', 'EL', 'MMC', 'SHW', 'BIIB']

stock_prices_2 = (
    yf.Tickers(symbols)
    .history(period="max", start="2013-01-07", end="2024-11-11")
    .Close
    .resample('1d')
    .ffill()
)

stock_prices_2

[*********************100%***********************]  96 of 96 completed


Ticker,AAPL,ABT,ACN,ADBE,ADP,AMGN,AMZN,AXP,BA,BAC,...,UNH,UNP,UPS,USB,V,VRTX,VZ,WFC,XOM,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-07 00:00:00+00:00,15.923544,26.518307,55.591919,37.939999,39.617489,64.115891,13.423000,50.671833,63.796135,9.715677,...,43.325710,50.158615,52.088001,22.549116,36.313915,47.290001,24.774508,24.654467,54.277870,NaN
2013-01-08 00:00:00+00:00,15.966393,26.526257,55.914715,38.139999,39.846153,63.840694,13.319000,50.959633,62.120159,9.627280,...,42.751801,50.123959,51.944744,22.583363,36.652008,48.500000,24.171621,24.611917,54.617386,NaN
2013-01-09 00:00:00+00:00,15.716858,26.701239,56.310112,38.660000,39.745262,64.217293,13.317500,51.010418,64.324089,9.185296,...,43.558586,50.397331,52.531452,22.788855,37.210876,48.250000,24.115534,24.611917,54.407497,NaN
2013-01-10 00:00:00+00:00,15.911680,26.923943,55.817860,38.619999,39.893230,63.594433,13.267000,51.459076,64.600601,9.466560,...,44.174091,50.474350,53.350113,23.042292,36.916477,46.799999,24.446426,25.101173,55.000088,NaN
2013-01-11 00:00:00+00:00,15.814120,26.652390,56.390816,38.090000,39.960472,62.978832,13.397000,51.840004,62.983292,9.346017,...,43.932892,50.428143,53.159096,22.946394,37.065990,47.310001,24.283781,24.888464,55.314922,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-04 00:00:00+00:00,221.766006,117.650002,343.899994,481.350006,288.365814,314.395813,195.779999,270.190002,155.070007,41.101368,...,555.638855,228.590988,131.383011,47.189999,291.850006,472.799988,41.060001,63.335064,118.610001,175.179993
2024-11-05 00:00:00+00:00,223.204422,117.730003,345.489990,486.420013,289.679016,316.756927,199.500000,275.940002,151.000000,41.648323,...,564.863525,232.528275,132.212799,47.610001,293.290009,499.880005,41.259998,63.702942,118.959999,175.270004
2024-11-06 00:00:00+00:00,222.475235,116.800003,356.130005,504.829987,303.606842,319.237122,207.089996,295.160004,147.160004,45.158794,...,594.410156,247.064407,132.291824,51.259998,307.399994,493.640015,41.160000,72.054817,121.000000,170.369995


In [4]:
stock_prices_2.to_csv("stock_prices.csv")